In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.linear_model import Ridge,ElasticNet,Lasso,LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [2]:
df = pd.read_csv('/Users/suryasaikadali/Downloads/pw_skills/end_to_End_projects/student_performance/artifacts/data.csv')

In [3]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race_ethnicity               1000 non-null   object
 2   parental_level_of_education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test_preparation_course      1000 non-null   object
 5   math_score                   1000 non-null   int64 
 6   reading_score                1000 non-null   int64 
 7   writing_score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [5]:
x = df.drop('math_score',axis = 1)
y = df['math_score']

In [6]:
cat_cols = [i for i in x.columns if df[i].dtype=='object']
num_cols = [i for i in x.columns if df[i].dtype=='int64']

In [7]:
cat_cols

['gender',
 'race_ethnicity',
 'parental_level_of_education',
 'lunch',
 'test_preparation_course']

In [8]:
num_cols

['reading_score', 'writing_score']

num_pipe = StandardScaler()
cat_pipe = OneHotEncoder(sparse_output=False,drop='first')

In [9]:
class Lab_enco:
    def __init__(self):
        self.le = LabelEncoder()
    def fit_transform(self,x):
        for i in cat_cols:
            x[i] = self.le.fit_transform(i)
        return x
    def transform(self,x):
        for i in cat_cols:
            x[i] = self.le.transform(i)
        return x

In [10]:
num_pipe = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)
                
cat_pipe = Pipeline(
    steps = [
        ('encoder',Lab_enco()),
        ('scaler',StandardScaler(with_mean=False))
    ]
)

In [11]:
preprocessor = ColumnTransformer([
    ('num',num_pipe,num_cols),
    ('cat',cat_pipe,cat_cols)
  
])
 

In [12]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=42)

In [13]:
x_train = pd.DataFrame(preprocessor.fit_transform(x_train))
x_test = pd.DataFrame(preprocessor.transform(x_test))

TypeError: Lab_enco.fit_transform() takes 2 positional arguments but 3 were given

In [ ]:
x_train

In [ ]:
models = {
    'linear regressor':LinearRegression(),
    'random forest Regressor':RandomForestRegressor(),
    'xgboost Regressor':XGBRegressor(),
    'ridge':Ridge(),
    'lasso':Lasso(),
    'elastic net':ElasticNet()
}

In [ ]:
for i,j in models.items():
    print(i)
    print('&&&&&&')
    print(j)

In [ ]:
def evaluate_model(true,predicted):
    mse = mean_squared_error(true,predicted)
    mae = mean_absolute_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2 = r2_score(true,predicted)
    return mae,rmse,r2

In [ ]:
r2_list = []

In [ ]:
for key,values in models.items():
    model = values.fit(x_train,y_train)
        
    y_pred = model.predict(x_test)

    mae,rmse,r2 = evaluate_model(y_test,y_pred)
    r2_list.append(f'{key}:{r2}')    
    print(f'The rsquare score of {key} model is {r2}')
    print(f'The mae  of {key} model is {mae}')
    print(f'The rmse of {key} model is {rmse}')
    print('%%%%%%%%%%%%%%%')
    print('\n')
        